### Posterior distributions

In [9]:
# add packages and load data

library(data.table)
library(tidyverse)
library(ggridges)
options(warn = -1)


cond <- "late"


df <- read.table(paste0('intercept',cond,'_post.txt'))

# two conditions of interest
conditions <- c("early", "late")

In [21]:
for (cond in conditions) {
    posteriors <- read.table(paste0('intercept',cond,'_post.txt'), header = TRUE)
    nobj <- dim(posteriors)[1]
    df.long <- posteriors %>%
          gather("ROI") %>%                                                 
          group_by(ROI) %>%
          mutate(p.plus = sum(value > 0)/length(value),
                 mean = mean(value),
                 median = median(value)) %>%                 
  arrange(p.plus, median) %>%
  ungroup() %>%
  mutate(index = rep(1:length(unique(ROI)), each = nobj),
         p.plus = format(round(p.plus, 3), nsmall = 2),
         p.plus.gray = p.plus,
         p.plus.gray = replace(p.plus.gray, p.plus.gray > 0.15 & p.plus.gray < 0.85, NA),
         p.plus.gray = as.numeric(p.plus.gray)) 
         
         
# create p+ dataframe that will mark all p+ values between 0.15 and 0.85 as NA, thus plotting gray distributions. This is done because ggplot does not allow for a range of values to be considered NA. Thus, a bit of a "cheat".

# Also formats P+ values so that all values are 3 decimals, even 0.

p.plus <- df.long %>%                                                         
  select(ROI, index, p.plus, p.plus.gray) %>%
  distinct()

# scaling for 2.3mm x 2.3mm per distribution
h <- length(unique(p.plus$ROI))*2.3
w <- length(unique(p.plus$ROI))*2.3 

ggplot(df.long, aes(x = value, 
            y = as.numeric(index),
            group = ROI, 
            fill = p.plus.gray)) +
  geom_vline(xintercept = 0, 
             linetype="solid",
             color = "black",
             alpha = .95, 
             size = .30) +
  stat_density_ridges(quantile_lines = TRUE,
                      quantiles = 2,
                      alpha = .95,
                      scale = 2.5,
                      color = "gray",
                      size = .35,
                      rel_min_height = 0.01) +
  scale_fill_gradientn(colors = c("#00ffff",
                                  "#00ccff",
                                  "#0069ff",
                                  "#0044ff",
                                  "#0000ff",
                                  "#000080", 
                                  "gray",
                                  "#99ff00",
                                  "#ffff00",
                                  "#ffcc00",
                                  "#ff9900",
                                  "#ff4400",
                                  "#cc1033"),
                       values = c(0, 0.025,
                                  0.02500000001, 0.050,
                                  0.05000000001, 0.075,
                                  0.07500000001, 0.10,
                                  0.10000000001, 0.125, 
                                  0.12500000001, 0.15,
                                  0.150000001, 0.85,
                                  0.850000001, 0.875,
                                  0.875000001, 0.90,
                                  0.900000001, 0.925,
                                  0.925000001, 0.950,
                                  0.950000001, 0.975,
                                  0.975000001, 1.0),
                        breaks = c(0, 
                                   0.025,
                                   0.050,
                                   0.075,
                                   0.100,
                                   0.125,
                                   0.150, 
                                   0.850,
                                   0.875,
                                   0.900,
                                   0.925,
                                   0.950,
                                   0.975,
                                   1)) +
  scale_x_continuous(breaks = c(-1, 0, 1),
                     labels = c( "Safe > Threat", "0", "Threat > Safe")) + 
  scale_y_continuous(breaks = c(1:length(p.plus$ROI)),
                     expand = c(0,0.1),
                     labels = p.plus$ROI, 
                     sec.axis = sec_axis(~.,
                                             breaks = 1:length(p.plus$ROI),
                                             labels = p.plus$p.plus)) +
    guides(fill = guide_colorbar(barwidth = 1.25,
                                barheight = 11.50,
                                nbin = 50,
                                frame.colour = "black",
                                frame.linewidth = 0.75,
                                ticks.colour = "black")) +
      theme(
        panel.background = element_blank(),
        panel.grid.major.y = element_line(color = "grey", size = 0.25),
        plot.title = element_text(size = 8,
                                  margin = unit(c(0.3,0,0,0.2),"cm"),
                                  face = "plain"),
        legend.title = element_text(size = 9),
        legend.text = element_text(size = 7),
        legend.position = c(.93, 0.20),
        axis.text.y = element_text(size= 7,
                                   color = "black",
                                   margin = unit(c(0,-0.05,0,0.05),"cm")),
        axis.text.y.right = element_text(size = 7,
                                         color = "black",
                                         margin = unit(c(0,0,0,-0.05),"cm")),
        axis.text.x = element_text(size = 9,
                                   color = "black",
                                   margin = unit(c(0.02,0,0,0),"cm")),
        plot.margin = unit(c(0,0.05,0,0.07), "cm"),
        axis.ticks.x = element_blank(),
        axis.ticks.y = element_blank()) +
      labs(x = NULL,
           y = NULL,
           fill = "P+")
ggsave(paste0("../images/",cond,"-posteriors.pdf"), width=w, height=h, units = "mm", dpi = 300)

}


Picking joint bandwidth of 0.0124
Picking joint bandwidth of 0.0101
